In [10]:
import pandas as pd
import scipy.stats as st
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
import math
import traceback
%matplotlib inline

In [11]:
# load data
df = pd.read_csv('health care diabetes.csv')
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [12]:
df_linear = pd.read_csv('Salary_dataset.csv')
df_linear

,Unnamed: 0,YearsExperience,Salary
0,0,1.2,39344.0
1,1,1.4,46206.0
2,2,1.6,37732.0
3,3,2.1,43526.0
4,4,2.3,39892.0
5,5,3.0,56643.0
6,6,3.1,60151.0
7,7,3.3,54446.0
8,8,3.3,64446.0
9,9,3.8,57190.0


In [13]:
# global variables
X = df.drop(columns=['Outcome']) # all input values
y = df['Outcome']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
print(f"X train: {X_train} Type: {type(X_train)}\n") # 2D array of np.float64
print(f"X test: {X_test} Type: {type(X_test)}\n")
print(f"y train: {y_train} Type: {type(y_train)}\n") # pd.Series array of ints
print(f"y test: {y_test} Type: {type(y_test)}\n")

X train:      Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
617            2       68             62             13       15  20.1   
101            1      151             60              0        0  26.1   
658           11      127            106              0        0  39.0   
558           11      103             68             40        0  46.2   
220            0      177             60             29      478  34.6   
..           ...      ...            ...            ...      ...   ...   
285            7      136             74             26      135  26.0   
27             1       97             66             15      140  23.2   
531            0      107             76              0        0  45.3   
534            1       77             56             30       56  33.3   
383            1       90             62             18       59  25.1   

     DiabetesPedigreeFunction  Age  
617                     0.257   23  
101                     0.17

C:\Users\ezrab\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [48]:
model = LogisticRegression()
model.fit(X_train, y_train)
print(f"Accuracy: {model.score(X_test, y_test)}")

Accuracy: 0.7760416666666666


C:\Users\ezrab\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [49]:
model.predict(X_test)

array([1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [53]:
# global variables
clusters = [] # stores x values of data
m_init = [1, 1, 1, 1, 1, 1, 1, 1]  # 8 predictor variable weights
b_init = 0
learning_rate = 0.01 # default
points = []
predictions = []
iterations = 500
prev_val = 0
epsilon = 0.001 # convergence test number, if y changes by a value less than ε then declare convergence
# h(x) = w1x1 + w2x2 + w3x3 + ... + w(n)x(n)

In [58]:
# supplemental functions

def sigmoid_function(x, m, b): # x is a vector of data
    y_temp_pred = b
    i = 0
    for val in x: # length of x == i
        y_temp_pred += val * m_init[i]
        
    return 1/(1 + math.exp(-1*y_temp_pred))

def linear_regression(x, m, b):
    y_temp_pred = b
    i = 0
    for val in x: # length of x == i
        y_temp_pred += val * m_init[i]
    print(y_temp_pred)
              
# optimize model using stochastic gradient descent

def cluster_points():
    # increment by some x and use a circle with radius x/2 to capture points
    print("Clustering points")
    increment = 10 # diameter of circle
    start = min(X_adjusted)
    end = max(X_adjusted)
    # (x - h)^2 + (y - k)^2 <= r^2
    radius = increment/2
    X_center = increment/2
    y_center = sigmoid_function(X_center, m_init, b_init)
    
    for i in range(start, end, increment):
        temp = []
        for j in range(start, end):
            if ((X_adjusted[j] - X_center)**2 + (y_adjusted[j] - y_center)**2) <= (radius)**2:
                temp.append(X_adjusted[j])
            
        if len(temp) > 0:
            clusters.append(temp)
        
        X_center = X_center + increment
        y_center = sigmoid_function(X_center, m_init, b_init)
        print(f"{X_center}, {y_center}")
        
        remove_duplicates()
    print(clusters)
    
def remove_duplicates():
    temp = dict()
    for i in range(0, len(clusters)):
        for j in range(0, len(clusters[i])):
            if clusters[i][j] not in temp.keys():
                temp[clusters[i][j]] = 1
            else:
                clusters[i].pop(j)
        
    
def random_cluster_points(): # empty range for randrange() (0,0,0)
    print("Selecting a random point per cluster")
    for cluster in clusters:
        if len(cluster) >= 1:
            if len(cluster) > 1:
                i = random.randint(0, len(cluster)-1)
                points.append(cluster[i])
            else:
                points.append(cluster[0])

    print(points)
    

# partial derivative of loss function with respect to slope, vector form
def derivative_m(x, y_pred, y):
    print("Calculating derivative m\n")
    print(f"x: \n{x} \ny: {y[0]} \ny_pred: {y}\n")
    return -2*x*(y[0]-y_pred)/len(weights) # returns a vector of data
    
    
# partial derivative of loss function with respect to y-intercept
def derivative_b(y_pred, y):
    return -2*(y[0]-y_pred)/len(weights)

# for scalar values only
def rms_error(predicted, actual):
    err = err + (predicted - actual)**2
    return math.sqrt(err/y_train.size)
    

def stochastic_gradient_descent(m, b):
    print("Performing SGD")
    sample = []
    err = 0
    
    for i in range(0, iterations):
        print(f"Iteration {i+1}")
        global m_init
        global b_init
        global prev_val
                
        rand_index = np.random.randint(0, X_train['Pregnancies'].size) # Used Pregnancies as placeholder category for # rows

        rand_val = X_train.iloc[rand_index]
        sample.append(rand_val)
        actual = model.predict(sample)
        # need to work on my own fit and prediction
        curr_val = sigmoid_function(rand_val, m_init, b_init)
        predictions.append(curr_val)
        print(f"Current value: {curr_val} | i: {i}\n | actual: {actual}\n")
        
        adjusted_m = derivative_m(rand_val, curr_val, actual)
        adjusted_b = derivative_b(curr_val, actual)
        print(f"Previous value: {prev_val} | Actual: {actual}\n")
#         err = err + rms_error()
        sample.clear()
        
        if (abs(curr_val - prev_val) < epsilon and i > iterations/2):
            print('Model converged!')
            break
        print(f"adjusted_m:\n{adjusted_m} | adjusted_b: {adjusted_b}\n")
        
        index = 0
        for weight in adjusted_m:
            m_new = m_init[index]
            m_new = m_new - weight * learning_rate
            m_init[index] = m_new
            index = index + 1
        
        b_init = b_init - adjusted_b * learning_rate
        print(f"New slope:\n {m_init}\n") 
        print(f"New intercept: {b_init}\n")
        print("---------------------------------")
        prev_val = curr_val

In [59]:
stochastic_gradient_descent(m_init, b_init) # m_adjusted = b_adjusted

Performing SGD
Iteration 1
Current value: 0.9283334905534407 | i: 0
 | actual: [0]

Calculating derivative m

x: 
Pregnancies                   8.00
Glucose                     133.00
BloodPressure                72.00
SkinThickness                 0.00
Insulin                       0.00
BMI                          32.90
DiabetesPedigreeFunction      0.27
Age                          39.00
Name: 61, dtype: float64 
y: 0 
y_pred: [0]

Previous value: 0.9475886869708705 | Actual: [0]

adjusted_m:
Pregnancies                  1.856667
Glucose                     30.867089
BloodPressure               16.710003
SkinThickness                0.000000
Insulin                      0.000000
BMI                          7.635543
DiabetesPedigreeFunction     0.062663
Age                          9.051252
Name: 61, dtype: float64 | adjusted_b: 0.23208337263836018

New slope:
 [-0.008493429149018935, -28.906181203546804, -21.277566258513705, -4.126957389255193, -19.640127812641673, -7.8312990450091

Iteration 95
Current value: 0.981605456978873 | i: 94
 | actual: [0]

Calculating derivative m

x: 
Pregnancies                   1.000
Glucose                     100.000
BloodPressure                72.000
SkinThickness                12.000
Insulin                      70.000
BMI                          25.300
DiabetesPedigreeFunction      0.658
Age                          28.000
Name: 446, dtype: float64 
y: 0 
y_pred: [0]

Previous value: 0.9999193010408254 | Actual: [0]

adjusted_m:
Pregnancies                  0.245401
Glucose                     24.540136
BloodPressure               17.668898
SkinThickness                2.944816
Insulin                     17.178095
BMI                          6.208655
DiabetesPedigreeFunction     0.161474
Age                          6.871238
Name: 446, dtype: float64 | adjusted_b: 0.24540136424471826

New slope:
 [0.011473833180527786, -28.739743803738182, -22.876978521577723, -4.076668821836692, -19.509574295419323, -7.899494542371597, 0


New slope:
 [0.003434553504221503, -29.196457987660807, -23.7028706499516, -4.345323496758734, -20.58974609737919, -8.159902634307358, 0.8890783393503195, -8.91436286015467]

New intercept: -0.34644242507086365

---------------------------------
Iteration 170
Current value: 0.6006737598641464 | i: 169
 | actual: [0]

Calculating derivative m

x: 
Pregnancies                   3.000
Glucose                     111.000
BloodPressure                62.000
SkinThickness                 0.000
Insulin                       0.000
BMI                          22.600
DiabetesPedigreeFunction      0.142
Age                          21.000
Name: 190, dtype: float64 
y: 0 
y_pred: [0]

Previous value: 0.04171689008813203 | Actual: [0]

adjusted_m:
Pregnancies                  0.450505
Glucose                     16.668697
BloodPressure                9.310443
SkinThickness                0.000000
Insulin                      0.000000
BMI                          3.393807
DiabetesPedigreeFunction 

Current value: 0.015803435729465506 | i: 251
 | actual: [1]

Calculating derivative m

x: 
Pregnancies                   9.000
Glucose                     112.000
BloodPressure                82.000
SkinThickness                24.000
Insulin                       0.000
BMI                          28.200
DiabetesPedigreeFunction      1.282
Age                          50.000
Name: 618, dtype: float64 
y: 1 
y_pred: [1]

Previous value: 0.8469119945304753 | Actual: [1]

adjusted_m:
Pregnancies                 -2.214442
Glucose                    -27.557504
BloodPressure              -20.176030
SkinThickness               -5.905179
Insulin                     -0.000000
BMI                         -6.938586
DiabetesPedigreeFunction    -0.315435
Age                        -12.302457
Name: 618, dtype: float64 | adjusted_b: -0.24604914106763362

New slope:
 [0.009867757596703603, -29.37267594771267, -24.625558174052454, -4.749025130899616, -21.906511779564745, -8.544662630251812, 0.89357561

In [57]:
# Tasks for now:
# logistic regression sklearn, compare my rms to built in ^
# ensure model has converged
# calculate root mean square error for each point ^
# convergence - rms error converges to smallest value it could achieve 

print(f"My Mean squared error formula:")
print(f"Sklearn Mean squared error: {mean_squared_error(model.predict(X_test), y_test)}")

My Mean squared error formula:
Sklearn Mean squared error: 0.22395833333333334
